In [14]:
# ======================================================
# 1) Introdução
# ======================================================
# Este notebook foi preparado para ser executado no Google Colab.
# Objetivo: treinar um modelo de Machine Learning (ex.: Rede Neural)
# para prever o valor mediano de casas na Califórnia.
#
# Dataset: California Housing (scikit-learn)
#
# Fluxo:
#   1. Carregar dataset
#   2. Dividir em treino/teste
#   3. Pré-processamento com ColumnTransformer
#   4. Baseline com DummyRegressor
#   5. Treinar seu modelo de ML (ex.: Rede Neural MLP)
#   6. Avaliar métricas (MAE, RMSE, R²)
#   7. Adicione PCA ao código e treine novamente o modelo de Rede Neural.

In [15]:
# ======================================================
# 2) Imports e Configurações
# ======================================================

import os
import random
import joblib
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor

# Semente para reprodutibilidade
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)



In [16]:
# ======================================================
# 3) Funções Auxiliares
# ======================================================

def load_california_housing():
    """
    Carrega o dataset California Housing como DataFrame.
    Retorna X, y, lista de colunas numéricas e categóricas.
    """
    ds = fetch_california_housing(as_frame=True)
    frame = ds.frame.copy()
    y = frame["MedHouseVal"]
    X = frame.drop(columns=["MedHouseVal"])
    numeric_features = X.columns.tolist()
    categorical_features = []  # não há colunas categóricas
    return X, y, numeric_features, categorical_features


def build_preprocessor(numeric_features, categorical_features):
    """
    Cria o pré-processador:
      - Numéricas: imputação de mediana + padronização
      - Categóricas: (não usado aqui, mas estrutura mantida para exemplos futuros)
    """
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
        ]
    )
    return preprocessor


def evaluate(model, X_test, y_test, label="Modelo"):
    """
    Avalia o modelo no conjunto de teste e retorna métricas (MAE, RMSE, R²).
    Compatível com versões antigas do scikit-learn.
    """
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))  # corrigido
    r2 = r2_score(y_test, preds)
    print(f"[{label}]  MAE={mae:.4f} | RMSE={rmse:.4f} | R²={r2:.4f}")
    return {"MAE": mae, "RMSE": rmse, "R2": r2}



In [17]:
# ======================================================
# 4) Carregar Dados e Split
# ======================================================

X, y, num_feats, cat_feats = load_california_housing()
print(f"Dataset carregado. X shape={X.shape} | y shape={y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED
)


Dataset carregado. X shape=(20640, 8) | y shape=(20640,)


In [18]:
# ======================================================
# 5) Baseline (DummyRegressor)
# ======================================================

preprocessor = build_preprocessor(num_feats, cat_feats)

model_pca = Pipeline(steps=[
    ("pre", preprocessor),
    ("pca", PCA(n_components=0.95, random_state=RANDOM_SEED)),  # mantém 95% da variância
    ("reg", MLPRegressor(
        hidden_layer_sizes=(12, 6),
        activation="relu",
        learning_rate_init=0.001,
        max_iter=800,
        random_state=RANDOM_SEED
    ))
])



In [19]:
# ======================================================
# 6) Seu Modelo de Machine Learning
# ======================================================
# Aqui você deve substituir pelo modelo que deseja treinar.
# Sugestão: Rede Neural com MLPRegressor, Random Forest ou Regressão Linear Ridge.

from sklearn.neural_network import MLPRegressor

model = Pipeline(steps=[
    ("pre", preprocessor),
    ("pca", PCA(n_components=0.95, random_state=RANDOM_SEED)),  # mantém 95% da variância
    ("reg", MLPRegressor(
        hidden_layer_sizes=(12, 6),
        activation="relu",
        learning_rate_init=0.001,
        max_iter=800,
        random_state=RANDOM_SEED
    ))
])

# Validação cruzada (5-fold)
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="r2")
print(f"Validação Cruzada R² (média ± desvio): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Treino final
model.fit(X_train, y_train)
metrics = evaluate(model, X_test, y_test, label="Rede Neural (MLP)")


Validação Cruzada R² (média ± desvio): 0.6853 ± 0.0035
[Rede Neural (MLP)]  MAE=0.4609 | RMSE=0.6455 | R²=0.6821


In [20]:
# ======================================================
# 7) Salvando Artefatos no Colab
# ======================================================

# Criar diretório para salvar resultados
os.makedirs("artifacts", exist_ok=True)

# Salvar modelo treinado
joblib.dump(model, "artifacts/model_california.joblib")

# Salvar métricas em arquivo texto
with open("artifacts/metrics.txt", "w") as f:
    for k, v in metrics.items():
        f.write(f"{k}: {v}\n")

print("✅ Artefatos salvos em ./artifacts")


✅ Artefatos salvos em ./artifacts
